In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.storage import LocalFileStore
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)
cache_dir = LocalFileStore("./.cache/")

memory = ConversationBufferMemory(
    llm=llm,
    max_token_limit=50,
    return_messages=True,
)

loader = UnstructuredFileLoader("./files/document.txt")
splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=600,
    chunk_overlap=100,
)
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir 
)

vectorstore= Chroma.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human", "{question}")
])

chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


Created a chunk of size 717, which is longer than the specified 600
Created a chunk of size 608, which is longer than the specified 600
Created a chunk of size 642, which is longer than the specified 600
Created a chunk of size 1444, which is longer than the specified 600
Created a chunk of size 1251, which is longer than the specified 600
Created a chunk of size 1012, which is longer than the specified 600
Created a chunk of size 1493, which is longer than the specified 600
Created a chunk of size 819, which is longer than the specified 600
Created a chunk of size 1458, which is longer than the specified 600
Created a chunk of size 1411, which is longer than the specified 600
Created a chunk of size 742, which is longer than the specified 600
Created a chunk of size 669, which is longer than the specified 600
Created a chunk of size 906, which is longer than the specified 600
Created a chunk of size 703, which is longer than the specified 600
Created a chunk of size 1137, which is lon

In [2]:
invoke_chain("Is Aaronson guilty?")

content='According to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [3]:
invoke_chain("What message did he write in the table?")

content='The message he wrote on the table was: "FREEDOM IS SLAVERY", "TWO AND TWO MAKE FIVE", and "GOD IS POWER".'


In [5]:
invoke_chain("Who is Julia?")

content='Julia is a character mentioned in the provided text. She is someone who the main character has strong feelings for and is willing to protect, even at the expense of his own well-being.'


In [7]:
def load_memory(_):
    return memory.load_memory_variables({})["history"]

load_memory(_)

[HumanMessage(content='Is Aaronson guilty?'),
 AIMessage(content='According to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'),
 HumanMessage(content='What message did he write in the table?'),
 AIMessage(content='The message he wrote on the table was: "FREEDOM IS SLAVERY", "TWO AND TWO MAKE FIVE", and "GOD IS POWER".'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content='Julia is a character mentioned in the provided text.'),
 HumanMessage(content='Who is Julia?'),
 AIMessage(content='Julia is a character mentioned in the provided text. She is someone who the main character has strong feelings for and is willing to protect, even at the expense of his own well-being.')]